# 【MMEngine重构】K210图像分类实战

## 1. 构建模型

In [1]:
from torch import nn
import torchvision
from mmengine.model import BaseModel
import torch

class MobileNet(BaseModel):
    def __init__(self):
        super().__init__()
        def conv_dw(in_channels, out_channels, kernel_size, strides, padding):
            return nn.Sequential(
                nn.Conv2d(in_channels, in_channels, kernel_size, strides, padding, groups=in_channels, bias=False),
                nn.BatchNorm2d(in_channels), nn.ReLU6(inplace=True),
                nn.Conv2d(in_channels, out_channels, kernel_size=1, bias=False), 
                nn.BatchNorm2d(out_channels), nn.ReLU6(inplace=True),
                nn.Conv2d(out_channels, out_channels, kernel_size=1, bias=False), 
                nn.BatchNorm2d(out_channels), nn.ReLU6(inplace=True))
        self.features = nn.Sequential(
            conv_dw(1,32,kernel_size=3,strides=2,padding=1),
            nn.MaxPool2d(2),
            conv_dw(32,64,kernel_size=3,strides=2,padding=1),
            nn.MaxPool2d(2),
            conv_dw(64,128,kernel_size=3,strides=1,padding=1),
            nn.MaxPool2d(2),
            conv_dw(128,10,kernel_size=3,strides=1,padding=1),
            nn.AdaptiveAvgPool2d((1, 1)),
            nn.Flatten(),
        )
    def forward(self, imgs, labels=None, mode='predict'):
        x = self.features(imgs)
        if mode == 'loss':
            return {'loss': nn.CrossEntropyLoss()(x, labels)}
        elif mode == 'predict':
            return x


## 2. 参数设置

In [2]:
# 超参数设置
epochs = 20
batch_size_train = 64
batch_size_test = 64
lr = 0.01
momentum = 0.5

# 训练配置
num_workers = 12
log_interval = 10
random_seed = 1
torch.manual_seed(random_seed)

## 3. 构建数据集和数据加载器

In [3]:
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchvision import datasets

train_dataloader = DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                  transform=transforms.Compose([
                      transforms.Resize(224),
                      transforms.ToTensor(),
                  ])),
    batch_size=batch_size_train, shuffle=True, num_workers=num_workers
)
test_dataloader = DataLoader(
    datasets.MNIST('../data', train=False, download=True,
                  transform=transforms.Compose([
                      transforms.Resize(224),
                      transforms.ToTensor(),
                  ])),
    batch_size=batch_size_test, shuffle=True, num_workers=num_workers
)

## 4. 构建评测指标

In [4]:
from mmengine.evaluator import BaseMetric

class Accuracy(BaseMetric):
    def process(self, data_batch, data_samples):
        score, gt = data_samples
        # 将一个批次的中间结果保存至 `self.results`
        self.results.append({
            'batch_size': len(gt),
            'correct': (score.argmax(dim=1) == gt).sum().cpu(),
        })

    def compute_metrics(self, results):
        total_correct = sum(item['correct'] for item in results)
        total_size = sum(item['batch_size'] for item in results)
        # 返回保存有评测指标结果的字典，其中键为指标名称
        return dict(accuracy=100 * total_correct / total_size)

## 5. 构建执行器并执行任务

In [5]:
from torch.optim import SGD
from mmengine.runner import Runner

runner = Runner(
    # 用以训练和验证的模型，需要满足特定的接口需求
    model=MobileNet(),
    # 工作路径，用以保存训练日志、权重文件信息
    work_dir='./work_dir',
    # 训练数据加载器，需要满足 PyTorch 数据加载器协议
    train_dataloader=train_dataloader,
    # 优化器包装，用于模型优化，并提供 AMP、梯度累积等附加功能
    optim_wrapper=dict(optimizer=dict(type=SGD, lr=lr, momentum=momentum)),
    # 训练配置，用于指定训练周期、验证间隔等信息
    train_cfg=dict(by_epoch=True, max_epochs=epochs, val_interval=1),
    # 验证数据加载器，需要满足 PyTorch 数据加载器协议
    val_dataloader=test_dataloader,
    # 验证配置，用于指定验证所需要的额外参数
    val_cfg=dict(),
    # 用于验证的评测器，这里使用默认评测器，并评测指标
    val_evaluator=dict(type=Accuracy),
)

#runner.train()

04/27 20:19:17 - mmengine - INFO - 
------------------------------------------------------------
System environment:
    sys.platform: win32
    Python: 3.10.11 | packaged by Anaconda, Inc. | (main, Apr 20 2023, 18:56:50) [MSC v.1916 64 bit (AMD64)]
    CUDA available: True
    numpy_random_seed: 295416231
    GPU 0: NVIDIA GeForce RTX 3060 Laptop GPU
    CUDA_HOME: None
    GCC: n/a
    PyTorch: 2.0.0
    PyTorch compiling details: PyTorch built with:
  - C++ Version: 199711
  - MSVC 193431937
  - Intel(R) Math Kernel Library Version 2020.0.2 Product Build 20200624 for Intel(R) 64 architecture applications
  - Intel(R) MKL-DNN v2.7.3 (Git Hash 6dbeffbae1f23cbbeae17adb7b5b13f1f37c080e)
  - OpenMP 2019
  - LAPACK is enabled (usually provided by MKL)
  - CPU capability usage: AVX2
  - CUDA Runtime 11.8
  - NVCC architecture flags: -gencode;arch=compute_37,code=sm_37;-gencode;arch=compute_50,code=sm_50;-gencode;arch=compute_60,code=sm_60;-gencode;arch=compute_61,code=sm_61;-gencode;arch=c

In [6]:
from tinynn.converter import TFLiteConverter
import os

checkpoint = torch.load('./work_dir/epoch_20.pth')
model = MobileNet()
model.load_state_dict(checkpoint['state_dict'])
model.eval()

dummy_input = torch.rand((1, 1, 224, 224))

output_path = os.path.join('./epoch_20.tflite')

# When converting quantized models, please ensure the quantization backend is set.
#torch.backends.quantized.engine = 'qnnpack'

# The code section below is used to convert the model to the TFLite format
# If you want perform dynamic quantization on the float models,
# you may refer to `dynamic.py`, which is in the same folder.
# As for static quantization (e.g. quantization-aware training and post-training quantization),
# please refer to the code examples in the `examples/quantization` folder.
converter = TFLiteConverter(model, dummy_input, output_path)
converter.convert()

INFO (tinynn.converter.base) Generated model saved to ./epoch_20.tflite
